<a href="https://colab.research.google.com/github/nsafaei/ICU-outcome-prediction/blob/master/Copy_of_DL_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 1. Download the dataset
#!wget https://user.engineering.uiowa.edu/~sbaek/files/cats_and_dogs.zip
!unzip -q cats_and_dogs.zip


# 2. Read the dataset
import os
import cv2
import numpy as np

from tqdm import tqdm

cat_list = ['PetImages/Cat/' + file for file in os.listdir('PetImages/Cat') if file.endswith('.jpg')]   # file paths for cat images
dog_list = ['PetImages/Dog/' + file for file in os.listdir('PetImages/Dog') if file.endswith('.jpg')]   # file paths for dog images

x = [] # empty list for storing images
y = [] # empty list for storing image labels. we will use one-hot encoding [1, 0] = cat, [0, 1] = dog

for f in tqdm(cat_list[0:100]): # this is how you use tqdm
# for f in cat_list: # this is without-tqdm version
  img = cv2.imread(f)  # read an image from the file list
  if img is not None:  # if image reading successful
    x.append(cv2.resize(img, (224,224)))  # append new image to the image list. resize and normalize before to append.
    y.append([1, 0])  # append new label to the label list
  
for f in tqdm(dog_list[0:100]):
  img = cv2.imread(f)
  if img is not None:
    x.append(cv2.resize(img, (224,224)))
    y.append([0,1])
x = np.array(x)
y = np.array(y)
#Ghanoon VGG
x = x - np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))



# 3. Train-test split
from sklearn.model_selection import train_test_split
import time # this is for generating random seed

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=int(time.time()))

print(len(x), len(train_x), len(test_x))

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


100%|██████████| 100/100 [00:00<00:00, 323.04it/s]


200 140 60


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
model = tf.keras.applications.VGG16(weights="imagenet", include_top=True,
                                    input_shape=(224,224,3)) # top means final decision layer
              

In [0]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
predicted_y = model.predict(test_x)

In [0]:
print(predicted_y) # this shows the probability distribution

[[2.2308324e-05 1.0397134e-05 4.8509678e-06 ... 7.1634154e-06
  8.3307586e-06 1.1711812e-04]
 [6.8217366e-07 1.5824247e-05 2.8228271e-06 ... 9.5753217e-07
  2.5745176e-04 1.8693842e-03]
 [6.3826229e-07 1.2648385e-06 4.4782223e-07 ... 7.6073678e-07
  1.7622703e-04 1.6836576e-04]
 ...
 [1.5858091e-06 1.5309297e-05 2.0700296e-04 ... 5.8953651e-07
  1.9244131e-04 2.0713102e-02]
 [3.3398041e-07 2.3531816e-06 3.2589240e-05 ... 7.5512196e-07
  1.1649994e-04 1.3256601e-02]
 [1.3431803e-05 7.5793350e-06 3.3229261e-07 ... 1.0418913e-05
  2.9360177e-04 2.2420259e-03]]


In [0]:
# number of images * 1000 classes
print(predicted_y.argmax(axis=-1)) #maximum axis along the last axis

[286 281 232 165 227 191 892 182 281 285 284 281 285 243 219 253 285 285
 285 247 283 285 285 222 282 151 162 235 282 171 250 700 185 283 248 282
 285 235 227 285 173 260 180 174 205 262 209 208 265 164 211 273 283 285
 794 285 281 285 285 208]


In [0]:
model1 = tf.keras.applications.VGG16(weights="imagenet", include_top=False,
                                    input_shape=(224,224,3))
X = model.get_layer('block5_pool').output

X = tf.keras.layer.Flatten()(X)
X = tf.keras.layers.Dense(512, activation = "relu")(X)
X = tf.keras,layers.Dense(2, activation="softmax")(X)

myModel = tf.keras.Model(model1.input, X)

myModel.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
model1 = tf.keras.applications.VGG16(weights="imagenet", include_top=False,
                                    input_shape=(224,224,3))
#fune tuning the last two layers for our new data set
for layer in model1.layers:
  #visit all layers in model.layers (VGG16 network)
  layer.trainable = False # freeze the ability to train the layers


X = model1.get_layer('block5_pool').output

X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(512, activation = "relu")(X)
X = tf.keras.layers.Dense(2, activation="softmax")(X)

myModel = tf.keras.Model(model1.input, X)

myModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
myModel.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizer.SGD(learning_rate=0.0001,momentum=0.9),
                metrics=["accuracy"])
myModel.fit(train_x, train_y, batch_size =32, epochs=5) #one-hat encoded y

In [0]:
model1 = tf.keras.applications.VGG16(weights="imagenet", include_top=False,
                                    input_shape=(224,224,3))
#fune tuning the last two layers for our new data set
for layer in model1.layers:
  #visit all layers in model.layers (VGG16 network)
  layer.trainable = False # freeze the ability to train the layers


X = model1.get_layer('block5_pool').output

X = tf.keras.layer.Flatten()(X)
X = tf.keras.layers.Dense(512, activation = "relu")(X)
X = tf.keras,layers.Dense(2, activation="softmax")(X)

myModel = tf.keras.Model(model1.input, X)

myModel.summary()